<a href="https://colab.research.google.com/github/Yeamin33/Inventory-Management-System/blob/main/Inventory_Management_System_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import os
from datetime import datetime

# File names
inventory_file = 'inventory.csv'
sales_file = 'sales.csv'

# Create Inventory file if not exist
if not os.path.exists(inventory_file):
    with open(inventory_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Item ID", "Item Name", "Category", "Quantity", "Unit Price", "Reorder Level"])

# Create Sales file if not exist
if not os.path.exists(sales_file):
    with open(sales_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Transaction ID", "Item ID", "Quantity Sold", "Sale Date", "Total Price"])

# Functions to manage inventory
def add_item(item_id, name, category, quantity, unit_price, reorder_level):
    with open(inventory_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([item_id, name, category, quantity, unit_price, reorder_level])
    print(f"Item {name} added successfully.")

def update_item(item_id, name=None, category=None, quantity=None, unit_price=None, reorder_level=None):
    items = []
    updated = False
    with open(inventory_file, mode='r', newline='') as file:
        reader = csv.reader(file)
        items = list(reader)

    for item in items:
        if item[0] == item_id:
            if name: item[1] = name
            if category: item[2] = category
            if quantity: item[3] = quantity
            if unit_price: item[4] = unit_price
            if reorder_level: item[5] = reorder_level
            updated = True
            break

    if updated:
        with open(inventory_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(items)
        print(f"Item {item_id} updated successfully.")
    else:
        print(f"Item {item_id} not found.")

def delete_item(item_id):
    items = []
    deleted = False
    with open(inventory_file, mode='r', newline='') as file:
        reader = csv.reader(file)
        items = list(reader)

    items = [item for item in items if item[0] != item_id]
    if len(items) < len(items):
        deleted = True
        with open(inventory_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(items)
        print(f"Item {item_id} deleted successfully.")
    if not deleted:
        print(f"Item {item_id} not found.")

# Stock level functions
def low_stock_items():
    low_stock = []
    with open(inventory_file, mode='r', newline='') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header row
        for item in reader:
            if int(item[3]) <= int(item[5]):  # Quantity <= reorder level
                low_stock.append(item)
    return low_stock

def out_of_stock_items():
    out_of_stock = []
    with open(inventory_file, mode='r', newline='') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header row
        for item in reader:
            if int(item[3]) == 0:  # Quantity is 0
                out_of_stock.append(item)
    return out_of_stock

# Sales recording functions
def record_sale(transaction_id, item_id, quantity_sold):
    items = []
    with open(inventory_file, mode='r', newline='') as file:
        reader = csv.reader(file)
        items = list(reader)

    for item in items:
        if item[0] == item_id:
            item_name, category, quantity, unit_price, reorder_level = item[1], item[2], int(item[3]), float(item[4]), item[5]
            if quantity >= quantity_sold:
                new_quantity = quantity - quantity_sold
                total_price = quantity_sold * unit_price
                update_item(item_id, quantity=new_quantity)

                # Log sale transaction
                with open(sales_file, mode='a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([transaction_id, item_id, quantity_sold, datetime.now().strftime("%Y-%m-%d %H:%M:%S"), total_price])
                print(f"Sale recorded: {quantity_sold} of {item_name} sold.")
            else:
                print(f"Insufficient stock for {item_name}.")
            return
    print(f"Item {item_id} not found in inventory.")

def sales_report():
    total_sales = 0
    item_sales = {}
    with open(sales_file, mode='r', newline='') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header row
        for row in reader:
            transaction_id, item_id, quantity_sold, sale_date, total_price = row
            total_sales += float(total_price)
            if item_id not in item_sales:
                item_sales[item_id] = 0
            item_sales[item_id] += float(total_price)

    print(f"Total Sales: ${total_sales}")
    print("Top Selling Items:")
    for item_id, total in item_sales.items():
        print(f"Item ID: {item_id}, Total Sales: ${total}")

# Inventory analysis functions
def analyze_inventory():
    fast_moving = []
    slow_moving = []
    category_revenue = {}
    total_value = 0

    with open(inventory_file, mode='r', newline='') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header row
        for item in reader:
            item_id, name, category, quantity, unit_price, reorder_level = item
            quantity = int(quantity)
            unit_price = float(unit_price)
            total_value += quantity * unit_price
            if quantity > 10:  # Fast moving
                fast_moving.append(name)
            elif quantity < 5:  # Slow moving
                slow_moving.append(name)

            if category not in category_revenue:
                category_revenue[category] = 0
            category_revenue[category] += quantity * unit_price

    print(f"Total Stock Value: ${total_value}")
    print(f"Fast-Moving Items: {', '.join(fast_moving)}")
    print(f"Slow-Moving Items: {', '.join(slow_moving)}")
    print("Revenue by Category:")
    for category, revenue in category_revenue.items():
        print(f"{category}: ${revenue}")

# Menu-driven user interface
def menu():
    while True:
        print("\nInventory Management System")
        print("1. Add Item")
        print("2. Update Item")
        print("3. Delete Item")
        print("4. View Low Stock Items")
        print("5. View Out of Stock Items")
        print("6. Record a Sale")
        print("7. View Sales Report")
        print("8. Analyze Inventory")
        print("9. Exit")

        choice = input("Enter your choice: ")

        if choice == "1":
            item_id = input("Enter Item ID: ")
            name = input("Enter Item Name: ")
            category = input("Enter Category: ")
            quantity = int(input("Enter Quantity: "))
            unit_price = float(input("Enter Unit Price: "))
            reorder_level = int(input("Enter Reorder Level: "))
            add_item(item_id, name, category, quantity, unit_price, reorder_level)

        elif choice == "2":
            item_id = input("Enter Item ID to update: ")
            name = input("Enter new Item Name (leave blank to skip): ")
            category = input("Enter new Category (leave blank to skip): ")
            quantity = input("Enter new Quantity (leave blank to skip): ")
            unit_price = input("Enter new Unit Price (leave blank to skip): ")
            reorder_level = input("Enter new Reorder Level (leave blank to skip): ")
            update_item(item_id, name or None, category or None, int(quantity) if quantity else None,
                        float(unit_price) if unit_price else None, int(reorder_level) if reorder_level else None)

        elif choice == "3":
            item_id = input("Enter Item ID to delete: ")
            delete_item(item_id)

        elif choice == "4":
            low_stock = low_stock_items()
            print("Low Stock Items:")
            for item in low_stock:
                print(item)

        elif choice == "5":
            out_of_stock = out_of_stock_items()
            print("Out of Stock Items:")
            for item in out_of_stock:
                print(item)

        elif choice == "6":
            transaction_id = input("Enter Transaction ID: ")
            item_id = input("Enter Item ID: ")
            quantity_sold = int(input("Enter Quantity Sold: "))
            record_sale(transaction_id, item_id, quantity_sold)

        elif choice == "7":
            sales_report()

        elif choice == "8":
            analyze_inventory()

        elif choice == "9":
            print("Exiting the program.")
            break
        else:
            print("Invalid choice! Please try again.")

# Run the menu
